


# Lab Course Machine Learning
# Exercise Sheet 11
February 2nd, 2021

Syed Wasif Murtaza Jafri-311226

## Exercise 0: Preprocessing Text Data

In [897]:
import numpy as np
import math
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from sympy import symbols, diff
import pandas as pd
import math
import warnings
import itertools
import matplotlib.colors as colors
from matplotlib.cm import ScalarMappable
from scipy.spatial.distance import pdist,cdist
warnings.filterwarnings('ignore')
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
import re
nltk.download("punkt")
from heapq import nlargest
import keyword
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wasif\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1212]:
categories = ['sci.med', 'comp.graphics']
newsgroups_train = fetch_20newsgroups(subset='all',categories=categories,shuffle=True,random_state=3116)
data = pd.DataFrame(data= np.c_[newsgroups_train['data'], newsgroups_train['target']],columns= ['txt','Y'])
data

,txt,Y
0,From: erikb@idt.unit.no (Erik Brenn)\nSubject:...,0
1,From: ab961@Freenet.carleton.ca (Robert Alliso...,1
2,From: grady@netcom.com (1016/2EF221)\nSubject:...,0
3,From: mark@ocsmd.ocs.com (Mark Wilson)\nSubjec...,0
4,From: vida@mdavcr.mda.ca (Vida Morkunas)\nSubj...,1
...,...,...
1958,Subject: Re: Trying to view POV files.....\nFr...,0
1959,From: brendan@gu.uwa.edu.au (Brendan Langoulan...,0
1960,From: Lawrence Curcio <lc2b+@andrew.cmu.edu>\n...,1
1961,From: lilley@v5.cgu.mcc.ac.uk (Chris Lilley)\n...,0


### 1) Preprocessing textual data to remove punctuation, stop-words

In [1213]:
# 1) Preprocessing textual data to remove punctuation, stop-words
data['txt'] = data['txt'].apply(lambda x: ' '.join([word for word in nltk.word_tokenize(re.sub(r'[0-9]', '', x).split('Subject:')[1].lower()) if word.isalnum() and word not in stopwords.words('english') and len(word)>2] ))

In [800]:
data

,txt,Y
0,graphics formats erikb erik brenn organization...,0
1,frequent nosebleeds robert allison organizatio...,1
2,hacker ethic organization capriccioso tin vers...,0
3,tiff philosophical significance silly organiza...,0
4,altitude adjustment summary adjust come organi...,1
...,...,...
1958,trying view pov files dane dane jasper organiz...,0
1959,input devices organization university western ...,0
1960,athlete heart organization doctoral student pu...,1
1961,graphics keywords alpha channel image lines or...,0


### 2) Implementing a bag-of-words feature representation for each text sample

In [801]:
# Calculating frequency of words in a document
data['f'] = data['txt'].apply(lambda x: ({k.upper(): sum(1 for _ in g) for k, g in groupby(x.split())}))
data

,txt,Y,f
0,graphics formats erikb erik brenn organization...,0,"{'GRAPHICS': 1, 'FORMATS': 1, 'ERIKB': 1, 'ERI..."
1,frequent nosebleeds robert allison organizatio...,1,"{'FREQUENT': 1, 'NOSEBLEEDS': 1, 'ROBERT': 1, ..."
2,hacker ethic organization capriccioso tin vers...,0,"{'HACKER': 1, 'ETHIC': 1, 'ORGANIZATION': 1, '..."
3,tiff philosophical significance silly organiza...,0,"{'TIFF': 1, 'PHILOSOPHICAL': 1, 'SIGNIFICANCE'..."
4,altitude adjustment summary adjust come organi...,1,"{'ALTITUDE': 1, 'ADJUSTMENT': 1, 'SUMMARY': 1,..."
...,...,...,...
1958,trying view pov files dane dane jasper organiz...,0,"{'TRYING': 1, 'VIEW': 1, 'POV': 1, 'FILES': 1,..."
1959,input devices organization university western ...,0,"{'INPUT': 1, 'DEVICES': 1, 'ORGANIZATION': 1, ..."
1960,athlete heart organization doctoral student pu...,1,"{'ATHLETE': 1, 'HEART': 1, 'ORGANIZATION': 1, ..."
1961,graphics keywords alpha channel image lines or...,0,"{'GRAPHICS': 1, 'KEYWORDS': 1, 'ALPHA': 1, 'CH..."


In [802]:
# Creating a dictionary which contains all words frequency in all rows by merging f column in above dataframe
dict1= {}
for index, row in data.iterrows():
    dict1 = {k.upper(): dict1.get(k, 0) + row['f'].get(k, 0) for k in set(dict1) | set(row['f'])}


In [803]:
# Creating dictionary with top 500 words maximum frequency
colList  = nlargest(500, dict1, key = dict1.get)
freqDict ={}
for i in colList:
    freqDict.update({i.upper():dict1.get(i)})


In [ ]:
freqDict

In [805]:
# creating feature columns from Frequency dictionary and putting values as Zero
for c in sorted(freqDict):
    data[c.upper()] = 0


In [806]:
data

,txt,Y,f,ABLE,ACCESS,ACTUALLY,ADDRESS,ADVANCE,ADVICE,AGO,...,WOULD,WRITE,WRITES,WRONG,WROTE,YEAR,YEARS,YES,YET,YORK
0,graphics formats erikb erik brenn organization...,0,"{'GRAPHICS': 1, 'FORMATS': 1, 'ERIKB': 1, 'ERI...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,frequent nosebleeds robert allison organizatio...,1,"{'FREQUENT': 1, 'NOSEBLEEDS': 1, 'ROBERT': 1, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,hacker ethic organization capriccioso tin vers...,0,"{'HACKER': 1, 'ETHIC': 1, 'ORGANIZATION': 1, '...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tiff philosophical significance silly organiza...,0,"{'TIFF': 1, 'PHILOSOPHICAL': 1, 'SIGNIFICANCE'...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,altitude adjustment summary adjust come organi...,1,"{'ALTITUDE': 1, 'ADJUSTMENT': 1, 'SUMMARY': 1,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,trying view pov files dane dane jasper organiz...,0,"{'TRYING': 1, 'VIEW': 1, 'POV': 1, 'FILES': 1,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1959,input devices organization university western ...,0,"{'INPUT': 1, 'DEVICES': 1, 'ORGANIZATION': 1, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1960,athlete heart organization doctoral student pu...,1,"{'ATHLETE': 1, 'HEART': 1, 'ORGANIZATION': 1, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1961,graphics keywords alpha channel image lines or...,0,"{'GRAPHICS': 1, 'KEYWORDS': 1, 'ALPHA': 1, 'CH...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [807]:
# Calculating bag of words for each word in the row
for index, row in data.iterrows():
    dict2 = row['f'] # frequency dictionary for that row
    for k in dict2:
        if k in data.columns:
            data.at[index,k] = dict2.get(k) # putting the frequency in the column
    
    

In [808]:
data

,txt,Y,f,ABLE,ACCESS,ACTUALLY,ADDRESS,ADVANCE,ADVICE,AGO,...,WOULD,WRITE,WRITES,WRONG,WROTE,YEAR,YEARS,YES,YET,YORK
0,graphics formats erikb erik brenn organization...,0,"{'GRAPHICS': 1, 'FORMATS': 1, 'ERIKB': 1, 'ERI...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,frequent nosebleeds robert allison organizatio...,1,"{'FREQUENT': 1, 'NOSEBLEEDS': 1, 'ROBERT': 1, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,hacker ethic organization capriccioso tin vers...,0,"{'HACKER': 1, 'ETHIC': 1, 'ORGANIZATION': 1, '...",0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,tiff philosophical significance silly organiza...,0,"{'TIFF': 1, 'PHILOSOPHICAL': 1, 'SIGNIFICANCE'...",0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
4,altitude adjustment summary adjust come organi...,1,"{'ALTITUDE': 1, 'ADJUSTMENT': 1, 'SUMMARY': 1,...",0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,trying view pov files dane dane jasper organiz...,0,"{'TRYING': 1, 'VIEW': 1, 'POV': 1, 'FILES': 1,...",0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1959,input devices organization university western ...,0,"{'INPUT': 1, 'DEVICES': 1, 'ORGANIZATION': 1, ...",0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1960,athlete heart organization doctoral student pu...,1,"{'ATHLETE': 1, 'HEART': 1, 'ORGANIZATION': 1, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1961,graphics keywords alpha channel image lines or...,0,"{'GRAPHICS': 1, 'KEYWORDS': 1, 'ALPHA': 1, 'CH...",0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [809]:
# removing extra columns and sorting columns
column = sorted(freqDict)
column.append('Y')
bowDf = data.reindex(columns=column) 
bowDf

,ABLE,ACCESS,ACTUALLY,ADDRESS,ADVANCE,ADVICE,AGO,ALGORITHM,ALLOW,ALMOST,...,WRITE,WRITES,WRONG,WROTE,YEAR,YEARS,YES,YET,YORK,Y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1961,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


### 3) Implementing a TF-IDF feature representation for each text sample

In [933]:
# creating feature columns from Frequency dictionary and putting values as Zero
for c in sorted(freqDict):
    data[c.upper()] = 0


In [934]:
data

,txt,Y,f,ABLE,ACCESS,ACTUALLY,ADDRESS,ADVANCE,ADVICE,AGO,...,WOULD,WRITE,WRITES,WRONG,WROTE,YEAR,YEARS,YES,YET,YORK
0,graphics formats erikb erik brenn organization...,0,"{'GRAPHICS': 1, 'FORMATS': 1, 'ERIKB': 1, 'ERI...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,frequent nosebleeds robert allison organizatio...,1,"{'FREQUENT': 1, 'NOSEBLEEDS': 1, 'ROBERT': 1, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,hacker ethic organization capriccioso tin vers...,0,"{'HACKER': 1, 'ETHIC': 1, 'ORGANIZATION': 1, '...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tiff philosophical significance silly organiza...,0,"{'TIFF': 1, 'PHILOSOPHICAL': 1, 'SIGNIFICANCE'...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,altitude adjustment summary adjust come organi...,1,"{'ALTITUDE': 1, 'ADJUSTMENT': 1, 'SUMMARY': 1,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,trying view pov files dane dane jasper organiz...,0,"{'TRYING': 1, 'VIEW': 1, 'POV': 1, 'FILES': 1,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1959,input devices organization university western ...,0,"{'INPUT': 1, 'DEVICES': 1, 'ORGANIZATION': 1, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1960,athlete heart organization doctoral student pu...,1,"{'ATHLETE': 1, 'HEART': 1, 'ORGANIZATION': 1, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1961,graphics keywords alpha channel image lines or...,0,"{'GRAPHICS': 1, 'KEYWORDS': 1, 'ALPHA': 1, 'CH...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [968]:
def calculateTfidf(dic,k,freqDict,lenDf):
    if(k in dic):
        tf = dic.get(k)/sum(dic.values()) # word frequqnce/ total words
        tf_idf = tf * np.log(lenDf/(df + 1))
        return tf_idf
    else:
        return 0
    
for i in sorted(freqDict): 
    df = len(data.query("`" +i+"`>0")) # for each word calculating df which is number of documents which contains that word
    data[i] = data.apply(lambda x: calculateTfidf(x['f'],i,freqDict,len(data)),axis=1) # calculating tf-idf for word and updating value of column
    
    

In [1009]:
# removing extra columns and sorting columns
column = sorted(freqDict)
column.append('Y')
tfDF = data.reindex(columns=column) 
tfDF

,ABLE,ACCESS,ACTUALLY,ADDRESS,ADVANCE,ADVICE,AGO,ALGORITHM,ALLOW,ALMOST,...,WRITE,WRITES,WRONG,WROTE,YEAR,YEARS,YES,YET,YORK,Y
0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0
1,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,1
2,0.0,0.00000,0.05414,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.05385,0.0,0.00000,0.0,0.00000,0.0,0
3,0.0,0.00000,0.00000,0.04644,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.04308,0.0,0.00000,0.0,0.04758,0.0,0
4,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.03658,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.00000,0.0,0.03033,0.0,0.00000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.04535,0.0,0.00000,0.0,0.00000,0.0,0
1959,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0
1960,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,1
1961,0.0,0.02075,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.00613,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0


### 4) Split the dataset randomly into train/validation/test splits according to ratios 80%:10%:10%

In [812]:
#Bag of words dataframe
bow_train = bowDf.loc[0:math.floor(len(bowDf)*0.8)]
bow_val = bowDf.loc[math.floor(len(bowDf)*0.8)+1:math.floor(len(bowDf)*0.9)]
bow_test= bowDf.loc[math.floor(len(bowDf)*0.9)+1:]

,ABLE,ACCESS,ACTUALLY,ADDRESS,ADVANCE,ADVICE,AGO,ALGORITHM,ALLOW,ALMOST,...,WRITE,WRITES,WRONG,WROTE,YEAR,YEARS,YES,YET,YORK,Y
1767,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1769,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1771,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1961,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [756]:
#Tf-idf Dataframe
tfDF_train = tfDF.loc[0:math.floor(len(tfDF)*0.8)]
tfDF_val = tfDF.loc[math.floor(len(tfDF)*0.8)+1:math.floor(len(tfDF)*0.9)]
tfDF_test= tfDF.loc[math.floor(len(tfDF)*0.9)+1:]


## Exercise 1: Implementing Naive Bayes Classifier for Text Data

In [1078]:
def implementingNaiveBayesBOW(df,testRow):
    classZeroPrior = len(df.query("Y=='0'"))/len(df) # prior probablity for Zero Class
    classOnePrior = len(df.query("Y=='1'"))/len(df) # prior probablity for One Class
    productProbClass0 = classZeroPrior
    productProbClass1 = classOnePrior
    for k in df.columns:
        if(k=='Y' ):
            break
        if(testRow[k]>0):
            colGivenClass0 = len(df.query("Y=='0' and `" +k+"`>0")) # number of documents in which word appeared given class 0
            colGivenClass1 = len(df.query("Y=='1' and `" +k+"`>0")) # number of documents in which word appeared given class 1
            totalColOcc = len(df.query(k+">0")) # number of documents in which word appeared
            productProbClass0 = productProbClass0 * (colGivenClass0/totalColOcc) # calculating probablity for word and producting them
            productProbClass1 = productProbClass1 * (colGivenClass1/totalColOcc)
    # normalizing product of probablities to make it under 1      
    naiveProbZero =  productProbClass0/(productProbClass0+productProbClass1)
    naiveProbOne =  productProbClass1/(productProbClass0+productProbClass1)

    if(naiveProbZero>naiveProbOne): #assigning class based on max probablity
        return '0'
    else:
        return '1'
    
            


In [1079]:
def predictNaiveBayesBow(trainDf,testDf):
    for index, row in testDf.iterrows():
        testDf.at[index,'Y_Hat'] = implementingNaiveBayesBOW(trainDf,row)
    return testDf

In [1080]:
resultDf = predictNaiveBayesBow(bow_train,bow_test)

In [1081]:
def calAccuracy(Y,Yhat):
    sum =0
    for i,j in zip(Y,Yhat):
        if(i==j):
            sum+=1
    return(sum/len(Y))*100

In [1082]:
accuracy = calAccuracy(resultDf['Y'].tolist(),resultDf['Y_Hat'].tolist())
print('Test accuracy:',accuracy)

Test accuracy: 90.81632653061224


In [1204]:
def implementingNaiveBayesTFIDF(df,row):
    classZeroPrior = len(df.query("Y=='0'"))/len(df)
    classOnePrior = len(df.query("Y=='1'"))/len(df)
    sumClass0 = 0
    sumClass1 = 0
    productProbClass0 = classZeroPrior
    productProbClass1 = classOnePrior

    for k in df.columns:
        if(k=='Y' or k=='Y_Hat' ):
            break
        if(row[k].item()>0):
            sumClass0 = sumClass0 + ((df.query("Y=='0'"))[k].sum()) # sum of all words tf-idf for class 0 
            sumClass1 = sumClass1 + ((df.query("Y=='1'"))[k].sum()) # sum of all words tf-idf for class 1 
            sumColTfClass0 = ((df.query("Y=='0'"))[k].sum())  # sum of k word tf-idf for all documents in class 0 
            sumColTfClass1 = ((df.query("Y=='1'"))[k].sum())  # sum of k word tf-idf for all documents in class 1 

            totalColOcc = len(df.query(k+">0")) # number of documents in which word appeared
            productProbClass0 = productProbClass0 * (sumColTfClass0/sumClass0)
            productProbClass1 = productProbClass1 * (sumColTfClass1/sumClass1)

    # normalizing product of probablities to make it under 1 
    naiveProbZero =  productProbClass0/(productProbClass0+productProbClass1)
    naiveProbOne =  productProbClass1/(productProbClass0+productProbClass1)
    #print('1',naiveProbZero,naiveProbOne)
    #print('2',naiveProbZero+naiveProbOne)
    if(naiveProbZero>naiveProbOne): #assigning class based on max probablity
        return '0'
    else:
        return '1'
    
            


In [1207]:
def predictNaiveBayesTfidf(trainDf,testDf):
    for index, row in testDf.iterrows():
        newDf = pd.DataFrame( columns = trainDf.columns)
        newDf = pd.concat([testDf.loc[index:index], trainDf], axis=0)
        testDf.at[index,'Y_Hat'] = implementingNaiveBayesTFIDF(newDf,testDf.loc[index:index])
        #print(row['Y'],testDf.loc[index:index]['Y_Hat'].item())
    return testDf

In [1208]:
resultDf = predictNaiveBayesTfidf(tfDF_train,tfDF_test)

In [1210]:
def calAccuracy(Y,Yhat):
    sum =0
    for i,j in zip(Y,Yhat):
        if(i==j):
            sum+=1
    return(sum/len(Y))*100

In [1211]:
accuracy = calAccuracy(resultDf['Y'].tolist(),resultDf['Y_Hat'].tolist())
print('Test accuracy:',accuracy)

Test accuracy: 78.57142857142857


## Exercise 2: Implementing SVM Classifier via Scikit-Learn

In [1061]:
# spliting dataset
X_train = bow_train.loc[:, bow_train.columns != 'Y'].to_numpy()
Y_train = bow_train[['Y']].to_numpy()

X_val = (bow_val.loc[:, bow_val.columns != 'Y']).to_numpy()
Y_val = (bow_val[['Y']]).to_numpy()

X_test = (bow_test.loc[:, bow_test.columns != 'Y']).to_numpy()
Y_test = (bow_test[['Y']]).to_numpy()

bow_test.shape

(196, 501)

In [1071]:
# applying SVM with poly kernal
svm = SVC(gamma=0.01,kernel='poly')
svm.fit(X_train, Y_train)
Y_hat_test = svm.predict(X_test)
calAccuracy(Y_test,Y_hat_test)

59.183673469387756

In [1063]:
def get_best_params(train_x, train_y,kernelRange,gamaRange):
    param_grid = {} 
    param_grid.update({'kernel':kernelRange})
    param_grid.update({'gamma':gamaRange})

    cv = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=42) 
    
    # for param grid calling GridSearchCV with penalty and diff hyper params in grid
    grid = GridSearchCV(SVC(random_state=3116), param_grid=param_grid, cv=cv,return_train_score=True, n_jobs=14, verbose=10) 
    grid.fit(train_x, train_y)

    print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))

    kernel_best = grid.best_params_['kernel']
    gamma_best = grid.best_params_['gamma']
    # creating model with best parameters
    svc = SVC(kernel =kernel_best,gamma=gamma_best,random_state=3116).fit(train_x, train_y)

    return svc,penalty_best,c_best

In [1064]:
kernelRange = ['linear', 'poly', 'rbf', 'sigmoid']
gamaRange =[0.09,0.1,0.12,0.13,0.14,0.15,0.2]
svc,penalty_best,c_best = get_best_params(X_train, Y_train,kernelRange,gamaRange)

<class 'list'>
Fitting 2 folds for each of 28 candidates, totalling 56 fits


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   4 tasks      | elapsed:    3.9s
[Parallel(n_jobs=14)]: Done  13 tasks      | elapsed:    5.5s
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    8.8s
[Parallel(n_jobs=14)]: Done  35 out of  56 | elapsed:   12.0s remaining:    7.2s
[Parallel(n_jobs=14)]: Done  41 out of  56 | elapsed:   13.4s remaining:    4.8s
[Parallel(n_jobs=14)]: Done  47 out of  56 | elapsed:   14.4s remaining:    2.7s
[Parallel(n_jobs=14)]: Done  53 out of  56 | elapsed:   15.7s remaining:    0.8s
[Parallel(n_jobs=14)]: Done  56 out of  56 | elapsed:   16.4s finished


The best parameters are {'gamma': 0.14, 'kernel': 'poly'} with a score of 0.93


In [1072]:
Y_hat_test = svc.predict(X_test)
accuracy = calAccuracy(Y_test,Y_hat_test)
print('Test accuracy:',accuracy)

Test accuracy: 88.77551020408163
